# Basic Tools and Agents

By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import datetime 

def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""

    now = datetime.datetime.now()  # Get current time
    return now.strftime("%I:%M:%S %p")  # Format time in H:MM AM/PM format

In [4]:
get_current_time()

'09:24:33 PM'

In [5]:
from langchain_core.tools import StructuredTool

time_tool = StructuredTool.from_function(
    name="Time",  # Name of the tool
    func=get_current_time,  # Function that the tool will execute
    # Description of the tool
    description="Useful for when you need to know the current time",
)

tools = [time_tool]

Create model

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

response = model.invoke([('human', 'Hi, what time is it?')])

response.content

"I'm sorry, I am a virtual assistant and I do not have the capability to know the current time. You can check the time on your device or ask a voice assistant like Siri or Google Assistant for the current time."

In [7]:
model_with_tools = model.bind_tools(tools)

In [8]:
response = model_with_tools.invoke([('human', 'Hi, what time is it?')])

print(f"Content: {response.content}")
print(f"Tool Calls: {response.tool_calls}")

Content: 
Tool Calls: [{'name': 'Time', 'args': {}, 'id': 'call_MkJuaQWp8emHHhj0RGBikfxc', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

## Create an Agent

In [21]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [23]:
response = agent_executor.invoke({"messages": [('human', 'Hi, what time is it?')]})

response["messages"]

[HumanMessage(content='Hi, what time is it?', id='739397a1-1084-437a-950f-9696436cca48'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_OGVhCP4A2VhNJwhBZ3uY6udI', 'function': {'arguments': '{}', 'name': 'Time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 56, 'total_tokens': 65}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3f177e86-24a9-4318-8139-fa4a7a927e3c-0', tool_calls=[{'name': 'Time', 'args': {}, 'id': 'call_OGVhCP4A2VhNJwhBZ3uY6udI', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 9, 'total_tokens': 65}),
 ToolMessage(content='09:32:09 PM', name='Time', id='fd9b1553-1bad-48c5-ab9e-99bd236104cb', tool_call_id='call_OGVhCP4A2VhNJwhBZ3uY6udI'),
 AIMessage(content='It is currently 09:32:09 PM.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_token

In [13]:
for chunk in agent_executor.stream(
    {"messages": [("human", "whats time is it?")]}
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YWeT7kEf3dTzWCr68gD45tZk', 'function': {'arguments': '{}', 'name': 'Time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 55, 'total_tokens': 64}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e4cc0e64-9f25-484a-991f-d6fe1b86beb6-0', tool_calls=[{'name': 'Time', 'args': {}, 'id': 'call_YWeT7kEf3dTzWCr68gD45tZk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 9, 'total_tokens': 64})]}}
---
{'tools': {'messages': [ToolMessage(content='09:26:03 PM', name='Time', tool_call_id='call_YWeT7kEf3dTzWCr68gD45tZk')]}}
---
{'agent': {'messages': [AIMessage(content='The current time is 09:26:03 PM.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 77, 'total_tokens': 90}

## Stateful Agent

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a thread_id when invoking the agent (so it knows which thread/conversation to resume from).  

In [14]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [29]:
agent_with_memory = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "0"}}

In [30]:
response = agent_with_memory.invoke(
    {"messages": [('human', 'Hi, what time is it?')]}, config
)

response["messages"]

[HumanMessage(content='Hi, what time is it?', id='0005a66d-b662-46c7-bff5-6200a8583d0b'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uEs68gOvLK3kO3XDaeAoSMp6', 'function': {'arguments': '{}', 'name': 'Time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 56, 'total_tokens': 65}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-43d6444a-6dd7-4d77-ac4b-caf769e9cfd8-0', tool_calls=[{'name': 'Time', 'args': {}, 'id': 'call_uEs68gOvLK3kO3XDaeAoSMp6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 9, 'total_tokens': 65}),
 ToolMessage(content='09:35:22 PM', name='Time', id='3cb0d90a-1566-4e73-af4b-3e6f121ba629', tool_call_id='call_uEs68gOvLK3kO3XDaeAoSMp6'),
 AIMessage(content='The current time is 09:35:22 PM.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_t

Now when we pass the same `thread_id`, the conversation context is retained via the saved state (i.e. stored list of messages

In [31]:
agent_with_memory.invoke(
    {"messages": [('human', 'What is 2 hours after time you have told before?')]}, config
)

{'messages': [HumanMessage(content='Hi, what time is it?', id='0005a66d-b662-46c7-bff5-6200a8583d0b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uEs68gOvLK3kO3XDaeAoSMp6', 'function': {'arguments': '{}', 'name': 'Time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 56, 'total_tokens': 65}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-43d6444a-6dd7-4d77-ac4b-caf769e9cfd8-0', tool_calls=[{'name': 'Time', 'args': {}, 'id': 'call_uEs68gOvLK3kO3XDaeAoSMp6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 9, 'total_tokens': 65}),
  ToolMessage(content='09:35:22 PM', name='Time', id='3cb0d90a-1566-4e73-af4b-3e6f121ba629', tool_call_id='call_uEs68gOvLK3kO3XDaeAoSMp6'),
  AIMessage(content='The current time is 09:35:22 PM.', additional_kwargs={'refusal': None}, response_metadata={'token_usage'